### 기본 세팅

In [1]:
import time
import pandas as pd
import re

from selenium import webdriver
import pyperclip
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

### MBTI 카페 크롤링

In [2]:
def crawling_mbticafe(menuid, mbti_group):
    
    # 드라이버 실행
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    
    
    
    # 네이버 로그인
    naver_id = 'bokju128'
    naver_pw = 'uks01uks28!'

    driver.get('https://nid.naver.com/nidlogin.login')             
    time.sleep(1)

    pyperclip.copy(naver_id)
    driver.find_element_by_xpath('//*[@id="id"]').click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    
    pyperclip.copy(naver_pw)
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)
    
    try:
        driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    except:
        driver.find_element_by_xpath('//*[@id="log.login"]').click()
    
    
    
    page = 0
    articleid = []     # id 리스트
    address   = []     # 주소 리스트
    nickname  = []     # 닉네임 리스트

    # 페이지를 이동하며 크롤링
    while page < 1:
        try:
            # 페이지 이동
            page += 1
            driver.get(f'https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.menuid={menuid}&search.boardtype=L&search.page={page}')
            try:
                driver.switch_to.frame('cafe_main')
            except:
                continue
                
                

            # id 크롤링
            currents = driver.find_elements_by_css_selector('#main-area > div > table > tbody > tr > td > div.board-number > div')
            
            for current in currents:
                articleid.append(current.text)



            # 주소 크롤링
            currents = driver.find_elements_by_css_selector('#main-area > div > table > tbody > tr > td > div > div > a.article')
            
            cur_address = []
            for current in currents:
                cur_address.append(current.get_attribute('href'))
            cur_address = cur_address[-15:]
            address = address + cur_address



            # 닉네임 크롤링
            currents = driver.find_elements_by_css_selector('#main-area > div > table > tbody > tr > td > div > table > tbody > tr > td > a')
            
            cur_nickname = []
            for current in currents:
                cur_nickname.append(current.get_attribute('onclick'))
            cur_nickname = cur_nickname[-15:]
            nickname = nickname + cur_nickname

            driver.switch_to.default_content()   
        except:
            pass
        
        
        
    # 닉네임에 mbti 포함 여부 판단
    nickname, drop_idx = find_mbti_in_nickname(nickname, mbti_group)
    articleid, address, nickname = drop_idx_in_address_and_nickname(articleid, address, nickname, drop_idx)
    
                
    
    title   = []     # 제목 리스트
    content = []     # 본문 리스트
    
    # 닉네임에 mbti가 있는 게시글만 크롤링
    for url in address:
        try:
            # 페이지 이동
            driver.get(url)
            try:
                driver.switch_to.frame("cafe_main")
                driver.sleep(1)
            except:
                pass
            
            
            
            # 제목 크롤링
            cur_title = driver.find_element_by_css_selector('#app > div > div > div > div > div > div > h3').text
            title.append(cur_title)
            
            
            
            # 본문 크롤링
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            currents = soup.select('div.article_viewer')

            cur_content = []
            for current in currents:
                cur_content.append(re.sub('\xa0', ' ', current.text.strip()))
            content.append(cur_content)
        except:
            pass
    
    
    
    data = pd.DataFrame.from_dict({'articleid': articleid, 'mbti': nickname, 'title': title, 'content': content}, orient='index')
    data = data.transpose()
    return data

In [3]:
def find_mbti_in_nickname(nickname, mbti_group):
    
    # 닉네임에 mbti가 있으면 추출
    for i in range(len(nickname)):
        nickname[i] = nickname[i].lower()
        
    drop_idx = []   
    for i in range(len(nickname)):
        check_cnt = 0

        for mbti in mbti_group:        
            if mbti in nickname[i]:
                nickname[i] = mbti

            if mbti not in nickname[i]:
                check_cnt += 1

        if check_cnt == len(mbti_group):
            drop_idx.append(i)
            
    return nickname, drop_idx



def drop_idx_in_address_and_nickname(articleid, address, nickname, drop_idx):
    
    # 닉네임에 mbti가 없으면 드랍
    for i in range(len(address)):
        while i in drop_idx:
            articleid.pop(i)
            address.pop(i)
            nickname.pop(i)
            drop_idx.remove(i)

            for idx in range(len(drop_idx)):
                drop_idx[idx] -= 1
                
    return articleid, address, nickname

### 파일 저장
출력 결과는 한 페이지씩 테스트한 것

In [4]:
# 닉네임에 아래가 포함되면 추출
mbti_group = ['intj', 'intp', 'entj', 'entp',
              'infj', 'infp', 'enfj', 'enfp',
              'istj', 'isfj', 'estj', 'esfj',
              'istp', 'isfp', 'estp', 'esfp',
              '인팁' ,'잇팁','엣팁','인프제',
              '엔프제', '잇프제', '엣프제', '인티제',
              '엔티제', '잇티제', '이티제', '엣티제',
              '에스티제', '엣프피', '엣픞', '잇프피',
              '잇픞', '엔팁', '엔프피', '엔픞',
              '인프피', '인픞']

In [5]:
# 크롤링 대상 게시판
boardnums = [72, 128, 101, 28, 18, 17, 16, 15, 14, 13, 12, 11]

In [6]:
for boardnum in boardnums:
    # 크롤링
    start_time = time.time()
    globals()[f'data{boardnum}'] = crawling_mbticafe(boardnum, mbti_group)
    print(f' menuid={boardnum} crawling time: {time.time()-start_time}')
    
    # 파일 저장
    datalen = len(globals()[f'data{boardnum}'])
    first_articleid = globals()[f'data{boardnum}']['articleid'].values[0]
    last_articleid = globals()[f'data{boardnum}']['articleid'].values[datalen-1]
    #globals()[f'data{boardnum}'].to_csv(f'data{boardnum}_from_{first_articleid}_to_{last_articleid}.csv')

 menuid=72 crawling time: 25.52163290977478
 menuid=128 crawling time: 23.065704345703125
 menuid=101 crawling time: 24.00944232940674
 menuid=28 crawling time: 29.714723587036133
 menuid=18 crawling time: 30.586082696914673
 menuid=17 crawling time: 32.329800844192505
 menuid=16 crawling time: 31.01547408103943
 menuid=15 crawling time: 28.169060230255127
 menuid=14 crawling time: 31.999090671539307
 menuid=13 crawling time: 30.362433195114136
 menuid=12 crawling time: 26.938995122909546
 menuid=11 crawling time: 24.505496978759766


In [7]:
data72.head(10)

,articleid,mbti,title,content
0,524617,infp,비싼옷 한벌 vs 싼옷 여러벌,[제가 지금 15만원짜리 맨투맨을 살까...차라리그돈으로 바지2개랑 맨투맨1 (이래...
1,524611,infj,최근에 너무 심하게 뚝딱 거렸어요 ;ㅁ; 뚝딱거리는 이유가 뭘까요?,[이거보다가 최근에 뚝딱거렸던거 생각나서 갑자기 부끄...ㅋㅋ​최근에 냥이 건강검진...
2,524607,entp,동물애호가??ㅡㅡ?,[전에..옆카페서 캣맘이 고양이 밥을 주는거에 관해논쟁을 한 일이 있엇는데(오늘은 ...
3,524605,entp,10월초까지 반팔반바지 입는 열많은체질 저뿐인가요,[저한테는 실질적으로 9월까지는 여름입니다....덥고 습함... 저같은사람 또 있을...
4,524578,infj,노동요로 재즈나 클래식을 들어요,[좀 특이하죠?ㅋㅋㅋ복잡한 심리를 차분하게 해준달까...]
5,524575,istp,ENTP의 드립력,[은 대체 어디서 나오는 거임?너무 기발함.웃겨 죽겠음!]
6,524573,infj,이런 날은 뭔가 음습하고 퇴폐적인 느낌의 음악이 땡겨요,[hooverphonic 의 'Mad About You' 같은 느낌혹시 비슷한 느낌...
7,524569,infp,내일은 슬픈 날이에요,[개강을 하다니..개강 멈추란 말이에요학생은 약해지는데 왜 개 강하는 거죠??개약 ...
8,524562,enfp,택시를 많이 안타봐서..,[인생 최대 7천원이상을 안타봤는데이만원을 찍었어여..불안해요 🥺]


In [8]:
data128.head(10)

,articleid,mbti,title,content
0,524583,istj,이거만 보면 유형뭐같나요??,[​]
1,524547,intj,대화 보면 이 두사람 유형 뭐 같나요? (스몰톡),[상황1. A가 코로나 선제검사 하러 주말 아침 일찍 갔는데 줄이 역대급이어서 줄이...
2,524518,엔팁,엔팁분들 직업이 뭔가요??!!,[궁금합니다!]
3,524309,intj,유형 감별 부탁드려요!!,[1. 친구 많고 완전 인싸 타입2. 동물 좋아하는데 특히 고양이🐱3. 잡지식 많음...
4,524274,intp,intp의 미디어 매체(영화 드라마)보는 특징일까요?,[1. 기본적으로 드라마 영화를 즐기진 않음​2. 하지만 드라마에 흥미가 생기면 ...
5,524227,엣프제,이 사람은 무조건 Exxx겠구나! 했던 분이,[인프피라네요...ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ인프피분들이... 원래 말도...


In [9]:
data101.head(10)

,articleid,mbti,title,content
0,524619,entp,왜이렇게들 남의 외모에 관심이 많으신지 모르겠어요,"[내가 살이 쪄서 살을 빼고 있든말든, 가을철에 반팔반바지를 입든말든, 타투를 하든..."
1,524596,intj,기능이 하나도 안 겹치는 유형들을 만났을 때 무슨 느낌을 받으시나요,"[INTJ - ISFJ, ESFJINTP - ISFP, ESFPINFJ - ISTJ..."
2,524477,intp,Intp 뒷담,[인팁은 뒷담들으면 진짜 신경 안써요? 전 인팁인데 신경쓰이던데...]
3,524449,enfp,잇프피와 이별,"[잇프피 남자친구와 이별했는데 잡고 싶어요.서로 다르게 생각하고, 갈등을 해결하는 ..."
4,524438,intj,남성분들 혹시,[다 맘속에 담아둔 여성 분 한명씩 있으신가요?아쉽게 헤어져서 시간이 지났음에도 계...
5,524307,intp,사람마다,[비슷한 사람에 끌리는 사람이 있고나랑 다른 부분에 매력을 느끼는 사람이 있잖아요이...
6,524252,enfj,척 하는 성격은 뭘까요?,[원래 알았던 척원래 그럴려고 했던척이미 다 아는척​누가봐도 몰랐고 누가봐도 내말에...
7,524201,isfp,이런 사람은 무슨 유형일까요?,[​​제 앞에서는 뒷담 잘까는데앞에 가서는 또 잘놀고 잘 치댄달까?무조건 좋게 지내...
8,524082,infj,여러분의 아침식사와 유형은?,[1. 아침 안 먹는다 - xxxx2. 브런치로 패스 - xxxx3. 국 밥 반찬 ...


In [10]:
data28.head(10)

,articleid,mbti,title,content
0,524610,intp,내가 많이 잘해준사람.나한테 많이 잘해준사람.이별후 어떤쪽이 더 생각날까요,[두쪽다경험있는데..둘다생각나네요.비오니 센치해짐근데확실히 전자쪽이 미련은없네요내가...
1,524553,intp,아쉬운사람이 먼저연락하게되네요,[카톡으로 서운한거얘기하고 남친이 잘할께라했는데 계속반복된문제라 그후서로 남친이랑 ...
2,524545,entp,전애인에게 새로운애인이 생겼을때 어떠신가요?,[저는 엔팁인데 ㅋㅋㅋㅋㅋㅋ​미련은 아닌데 그냥 기분은 썩 좋지 않네요 ㅎ........
3,524506,enfj,INFJ 남자의 관심표출법,[INFJ남자의 관심표출법이 어떻게되나요?]
4,524394,intp,점 믿으세요?,[유명한곳가서 남친꺼랑 궁합봤는데.(역학아니구 신점)저의비밀을 맞추더라구요.아무도모...
5,524382,entp,문득 궁금해진 가족분위기?!,[제가 대충 파악한 바로는...​​​아빠 ab형 isfp 엄마 a형 isfj첫째딸 ...
6,524378,intp,출출해용,[곱창전골에 소주마시고싶어요이자카야가서 먹는거좋아했는데 새벽까지안하니 재미가없네요]
7,524326,infp,제꺼 어떤 유형인거 같으세요??,[엠비티아이중 어떤 걸까요?]
8,524304,isfp,잇프피 질문 받아요!,[ISFP 여자 질문 받습니다..... 부끄럽지만 정성스레 답할게요 :---)]


In [11]:
data18.head(10)

,articleid,mbti,title,content
0,524572,esfj,"인프피의 연락 텀 ,, , , ,, , 하",[썸? 뭐 그런 사이인데 (왜냐면 확신이 좀 없기 때문 ㅠㅠ) 연락텀이 빠를 땐 거...
1,524513,infp,.,[도서관은 내일 한번 오랫만에 가보려고요.어제 갔다가 휴관일인거 알고 돌아왔어요.여...
2,524502,istp,INFP(인프피) 남자분들 잠수이별 궁금해요,"[남자친구는 INFP(나이는 40대)인데요,100일 좀 넘게 사겼고 장거리 연애로 ..."
3,524458,intj,인프피랑 싸우면 길게가는 감정의 골!!,"[저 남자들이 연애하다 싸우게될때여자들한테 맞춰주고, 사과하고 하다가질리는 그감정 ..."
4,524440,enfj,인프피님들 어제 일이후로 불편한데 어쩌죠,[얘기하기 조심스럽고 본인감정을 잘얘기안하니 갑자기 손절할까 하는 두려움에 불편해졌...
5,524407,entj,엔프피 연락,[조금 심각한 문제로 얘기중인데 상대방은 엔프피입니다원래 엔프피는 생각이 아직 마무...
6,524392,enfp,백색소음 틀고 자려고 누웠는데,[무슨 쥬라기 공원 온 것 같네요 프테라노돈 날아다니며 소리지르는 소리나요​다들 안...
7,524385,enfj,infp 남자는 대화를 어려워하나여..?,[갈등이 생기면 시간을 둬야하는걸로 아는데그러다 혼자 손절치면 어떻게하죠?​저를 좋...
8,524370,infp,인프피의 가장 큰 적은 '감정 기복'이라고 생각해요,[​인프피에게 가장 큰 적은 무엇일까요?개인적으론 '감정 기복'이라고 생각합니다.​...
9,524364,entp,# INFP 사촌 관찰기,"[1. 인생에 관해 비관적이다. 나도 비관적이긴 한데,이친구 만큼은 아니다.. 인..."


In [12]:
data17.head(10)

,articleid,mbti,title,content
0,524618,istj,인팁 분들 저한테 관심있는건지 한 번만 봐주세요 ㅠㅠ,[저는 잇티제고 상대방(남자) 인팁이에요!! 썸타다가 한 번 깨졌던 사이인데요 설명...
1,524614,entp,"방학 마지막날, 너무너무 심심한데 ENTP 여자사람한테 질문 있으신분들 마구 해주세요","[심심하니까 질문 받아보겠습니다 TMI, 고민상담 등등 전부 가능]"
2,524585,entp,INTP ~ INFP ~ ENTP,[mbti form q 검사결과ENTPbut 과거 INTP였던 것 같다 라는 말을 ...
3,524503,esfj,인팁이 살면서 기억에 남는 일,[제가 다른 커뮤에 제 경험이 담긴 이 글을 쓰다가 이거 여기에 쓸 글이 아니라 엠...
4,524486,infj,ENTP 분들 혹시 삶에 회의감이 느껴질 때,[무기력해져서 현생이 지치다가도 종종 미래가 막막해질 땐 어떻게 극복하시는가요?​실...
5,524481,인팁,자가격리중이에요 ...,[가족중 한명이 양성나와서ㅠㅠㅠㅠ집에서 자가격리중인데 답답하네요 재택근무중임니다.....
6,524475,entp,엔팁 좋아하시는분 계신가요?,[엔팁의 어떤 성향을 조아하시는건지 궁금해요또 엔팁의 싫은점도 좀 알려주시면 감사하...
7,524468,인팁,entp의 모순,[지금 보닌 상태))\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n...
8,524350,intp,인팁 감수성 많지 않나요?,[기계라고 하던데..ㅋㅋㅋ 저는 솔직히 공감능력은 별로 없는 편인데 감수성은 많은 ...
9,524330,infj,intp 연애할 때,[인팁이랑 연애 초반인데 이제 막 일주일 넘어가거든요..이 깨가 쏟아지는 시기? 서...


In [13]:
data16.head(10)

,articleid,mbti,title,content
0,524576,esfp,오늘 문득 드는 생각이 esfp는 금사빠 금사식이 맞을까요?,[전 어릴때 잇프피였다가.. (오래전이라 mbti는 없었지만) 현재 외향적이고 활달...
1,524524,infp,"잇프피의 성격,특징,팩폭",[]
2,524429,esfj,ISFP 잇프피 분들 질문이요~,[안녕하세요! 저는 잇프피남을 짝사랑하고 있는 ESFJ 여자 입니다~​5번쨰 약속을...
3,524405,인프피,잇프피여 엣프피남,[잇프피 여자입니당 연애안한지 5년이넘어가는데요ㅋㅋ전너무 기차니즘에 계획이있어야 나...
4,524329,isfp,잇프피 여러분 어떤 댕댕이가 나오시나요,[daengdaengmbti.netlify.app\n\n\n\n \n\n\n\n\n...
5,524314,infp,잇프피의 연애스타일,[저한테는 스타일이 잘맞네요~다른 잇프피분들도 비슷한가요?]
6,524192,entp,isfp인 친구에게,[​연인이든 친구든몸이멀어지면 마음도 멀어져.이말들음 어떠실까요?ㅠㅠ]
7,523904,istp,Isfp 원래 고집 쎄요??,[진짜 말도 안 되는 고집으로 계속 우기면서 말하는데 진짜 설득이 안돼요.. 표정 ...
8,523894,isfp,isfp)핑프 어떻게 생각 하세요?,"[그렇게 티는 많이 안내지만,,개인적 핑프 너무 싫어해요.제 엠비티아이 다른 분들도..."
9,523847,intp,과외학생이 ISFP인데요,[과외하는 학생이 isfp인데 자꾸 아재개그를 던져요. 언어유희라고 부르기도 민망한...


In [14]:
data15.head(10)

,articleid,mbti,title,content
0,524591,esfj,나는야 인팁 만나는 엣프제~,[ISTP 남자와 1년째 만나고 있는 ESFJ/ENFJ 입니다* 글이 좀 길어요! ...
1,524589,enfp,istp 남성분들 찐여사친/호감좋아함,[행동이나 말이 어떻게 다르시나요?]
2,524433,enfj,유사성향 vs 반대성향,[예전 연애는 성향이 비슷해서 티키타카 잘되고 불꽃ㅋㅋ처럼 타오르고 뭐 그랬는데엡....
3,524401,인팁,잇팁분들,[잇팁분들 친한 친구가 만약에 나랑 아예 관련 없는 친구한테 내가 말해준 나의 개인...
4,524339,enfp,잇팁 남친 이런 상황 좋아했을까용,[얼마전에 남친이 술을 먹다가 잔뜩 취해가지고 저한테 영상통화를 걸었더라구요.​참고...
5,524236,enfp,istp 살갑게 굴다가 갑자기 차갑게 굴면,[제가 뭐 잘못한 걸까요?어제까진 잘 놀았는데..​처음엔 한명만 그러길래 원래 그런...
6,524226,estp,estp 20대 여자에게 궁금한거 있으신가욥,[​]
7,524098,intp,ISTP 남 결혼,[둘 다 결혼 적령기이지만 엄청 급한나이는 아니고요 사귄지는 1년 정도.남친이 완전...
8,523900,enfp,잇팁분들 생각 어떠신가요?ㅜㅜ,[잇팁 남친과 만나고있는데 ​며칠 전 데이트는 너무 잘했어요!!​근데 카톡스타일이 ...
9,523722,esfj,잇팁여성분들 .. 지나가다 한번 꼭 봐주세요..,[제가 잇팁 친구랑 한달째 연락을 이어오던 중 잇팁과 만나자는 약속을 잡았었습니다....


In [15]:
data14.head(10)

,articleid,mbti,title,content
0,524613,infj,INFJ 현자타임 주절주절,[비가 또르르 내리고 날씨도 적적하여처음으로 푸념 글을 적네요 XD​자기보다 연하이...
1,524369,infj,"우울증 비율이 유독 인프제, 인프피가 높은 것 같네요",[뭐 불건강하면 모든 유형이 우울하긴 하겠지만​여기 글 올라오는 것들 쭉 읽어보니 ...
2,524357,infj,재간둥이의 백신 1차 접종 tmi 후기(첫쨋날 0830),[오늘 백신 맞고 왔어요!저는 20대라서 10부제 예약해서 오늘 맞고 온건데 10부...
3,524294,infj,"INFJ 본인, 요즘 회사에서 자주 듣는 말","[1.회사에서 네가 젤 웃기다2.젊어보이는데 대화 해보면 어른스럽다, 애늙은이 소리..."
4,524264,infj,INFJ의 시선에서 본 찐 ENFJ.,[안녕하세요 혼냐입니다.​오늘 하루도 잘 보내셨나요? 오늘도 수고 많으셨어요.​오늘...
5,524224,infp,김종국 씨가... INFJ ???,[https://www.youtube.com/watch?v=vZzwYyc6wWk\n...
6,524179,entp,엔팁 여친 사고침,[​엔프제 남친이랑 싸울 일이 있었는데​한동안 잘 지내다가 또 직설적으로 하고싶은 ...
7,524102,isfp,내용펑해요!,[내용펑해요!]
8,524075,intj,펑,[펑해요 ㅠㅠ 감사합니다]
9,523967,esfj,인프제 남자분들 도와주세요 ㅠ,[안녕하세요. 인프제남의 심리가 너무 어려워서 여쭈어요.​저는 인프제남과 어느정도 ...


In [16]:
data13.head(10)

,articleid,mbti,title,content
0,524595,esfp,내 잇프제는요,[​(개심각하게) 무뚝뚝하지만 다정하고엄청 잘생겼구!! 옷도 센스입게 잘입고 엄청 ...
1,524581,esfp,잇프제 질문이요,[평소 자주 연락하고 주말에도 만났던 사이인데 관심있다 좋다 표현하던사람이연락뜸하거...
2,524567,esfj,엣프제 30초 여자 질문받아요!,[연애 성격 아무거나 물어보세요 >_<오늘 일이 많이 없구ㅋㅋㅋ엠비티아이 카페 너무...
3,524496,esfj,일하기 싫어영~ 엣프제 질문 받아용~,[엣프제 여자에게 궁금하신거 물어봐주세용~​일중인데 일하기 싫어영 ~ 헝]
4,524442,infj,매일 선톡이 오던 잇프제남!,"[썸타기 시작하고, 잇프제남이 매일 출근길에 선톡을 보내주었는데요! 혼자서 여행을 ..."
5,524436,엔팁,ESFJ 미련,[보통 엣프제는 연애할 때 최선을 다했기 때문에 빨리 잊는 편이라 하잖아요그럼 연애...
6,524431,esfp,잇프제 선톡 말인데요,[좋아하는사람이 잇프제인데선톡 받거싶거둥요?근데 제가 출근시간때문에 새벅에 일어나고...
7,524420,엣프제,엣프제분들!!,[취미활동 있으세용??가끔 혼자있게 될때ㅋㅋㅋㅋㅋ (혼자있는거 싫어함...)혼자 있...
8,524371,isfj,T성향 강해진 잇프제 질문해주세여,[이따 1시간뒤에 남자친구 깨워주기로 해서안자고 있는데 아무나 질문 해주세요ㅠㅠㅠ]
9,524324,isfj,잇프제친구가 내편만 들어줘서 좋다고 생각했었는데,[더 정확히 말하면 공감능력만 있으면 최고 좋은 친구다라고 생각하고 살았었어요내편...


In [17]:
data12.head(10)

,articleid,mbti,title,content
0,524599,esfp,불건강 또는 어장하는 엔티제에게 어장당한 후기,"[​​처음이야 호기심에 호감에 나에게 무작정 들이대던 놈이었지만진지하게 만나자, 사..."
1,524565,entj,엔티제가 상대를 못 잊는 이유,[상대에게 잊는 정 없는 정 다 털려놓고도 잊지 못하는 이유는 뭘까요]
2,524537,entj,intj 남자..궁금해요,[전 entj라서 카톡하면서 광장히 비슷하다 생각했어요.그래도 뭔가 모르게 다른 부...
3,524516,istp,entj 인간관계 질문 (질문이 겁나 많아요),[1. 싫어하는 사람과 잘 지낼 수 있는지2. 친구를 가리며 사귀는데 가리는 기준은...
4,524511,entj,사고의 문을 스스로 닫아버린 특정 공무원 상대는 늘 어렵네요.,[350억 예산으로 고용창출 600명의 특구 지정...?은 좀 이해가 안가지만 쨌든...
5,524488,인티제,인티제님들,[재밌게 보신 영화 있나요? 오늘 시간적 여유가 되서 영화를 볼려고 하는데재밌게 본...
6,524487,infp,인티제 여러분들은...,[원래 장난기 많으시고... 칭찬 많이 해주시는 편이고.... 다정한 말 잘 해주시...
7,524395,intj,역시 인티제 최고의 애정표현은,[자기 할 일 다 던져놓고 연인 보러 가는 거 아닐까요?]
8,524388,intj,Intj끼리 연애하면,[​어떤 일이 벌어질까요. 어떤 형태일까요. 어떤 양상일까요.]
9,524376,intp,엔티제님덜... 만남의 이유,[만남의 이유가 그렇게 중요한가요?저는 남녀가 만나는 데 이유는 핑계에 불과하다고 ...


In [18]:
data11.head(10)

,articleid,mbti,title,content
0,524325,istj,ISTJ 여자 질문 받아용,[연애 중이라 연애 관련 질문도 환영합니당 ㅎㅎㅎ]
1,524251,esfj,Istj 분들 봐주세용,"[Istj 분이랑 소개팅을 했는데요,,다음날 카톡하다가 자연스럽게 말을 놓더라구요(..."
2,524061,istj,ISTJ's burden : 잇티제식 책임과 의무,[ISTJ들이 보통 책임감과 의무감에 민감하고 자신이 그것을 짊어져야 한다는 생각이...
3,524028,isfj,ESTJ 썸남이랑 카톡하는데,"[너무,,할말 없게보내요,,;;;;씹을수도 없고,,,​근데 지금 본인 뭐하고 있는지..."
4,523783,enfp,"불건강 잇티제=제 전남친, 건강 잇티제=제 베프 예요",[여기에 잇티제남이랑 잘됐다고 글쓰고 그랬는데 ㅜ사귀어보니 정말 잇티제의 단점이 극...
